In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import platform
import matplotlib as mpl
import random
from copy import copy
import re

import matplotlib.animation
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sp

from pathlib import Path

from relnet.agent.baseline.baseline_agent import *
from relnet.evaluation.storage import EvaluationStorage
from relnet.evaluation.experiment_conditions import *
from relnet.evaluation.file_paths import FilePaths
from relnet.visualization import *

from relnet.state.network_generators import *
from relnet.state.geometric_network_generators import *

generators_considered = [GeometricInternetTopologyNetworkGenerator, GeometricMetroNetworkGenerator]
fp = FilePaths('/experiment_data', 'aggregate', setup_directories=False)

# Table with basic graph properties 

In [ ]:
props_rows = []

for gen_class in generators_considered:
    gen = gen_class(**RealWorldNetworkGenerator.get_default_kwargs())
    gen_name = gen.name
    for i in range(gen.num_graphs):
        entry = {}
        if i == 0:
            entry['gen'] = gen_name
        else:
            entry['gen'] = ""
        entry['graph_name'] = gen.graph_names[i]
        for k, v in gen.graph_props[i].items():
            entry[k] = v
        
        props_rows.append(entry)

cols_order = ['gen', 'graph_name', 'num_nodes', 'num_edges']
props_df = pd.DataFrame(props_rows)
props_df = props_df[cols_order]
props_df.reset_index(drop=True, inplace=True)
props_df.head(20)

In [ ]:
props_df.replace(network_generator_display_names, inplace=True)
filename = 'rw_graph_metadata.tex'
texfile =  str(fp.figures_dir / filename)
fh = open(texfile, 'w')
table_colformat = f"cccc"
props_df.to_latex(buf=fh, column_format=table_colformat, index=False)
fh.close()

In [ ]:
replace_dict = {
        r"gen": r"Dataset",
        r"graph\\_name": r"Graph",
        r"num\\_nodes": r"$|V|$",
        r"num\\_edges": r"$|E|$",
    }

with open(texfile, 'r') as f:
    raw_content = f.read()

processed_content = raw_content
for orig, targ in replace_dict.items():
    processed_content = re.sub(orig, targ, processed_content, flags = re.M)

with open(texfile, 'w') as g:
    g.write(processed_content)